In [1]:
# KNOWN ISSUES: If ACM has document open, will stall this procedure
from cyschoolhousesuite import *
import xlwings as xw

sch_ref_path = "Z:\\ChiPrivate\\Chicago Data and Evaluation\\SY18\\SY18 Team Placement\\SY18 FINAL Team Placements.xlsx"
sch_ref_df = pd.read_excel(sch_ref_path, sheetname="School Data")
sch_ref_df = sch_ref_df[pd.notnull(sch_ref_df['School'])]

CP_df = get_report("00O1a000002rm2S")

CP_df = CP_df[CP_df['Program: Program Name'].str.contains("Attendance")]

CP_df.sort_values(by=['Account Name', 'Student: Student Name'], inplace=True)

cols = ['Account Name',
        'Student: Student Name',
        'Student Id',
        'Student: Grade',
        'Intervention Primary Staff: Staff Name']

CP_df = CP_df[cols]

# Update existing files
for x in sch_ref_df['cyconnect Folder'].unique():
    try:
        file_path = ("Z:\\" + x + "\\" + "SY18 Attendance Tracker - "
                     + sch_ref_df['Informal School Name'][sch_ref_df['cyconnect Folder'] == x].values[0] + ".xlsx")
        wb = xw.Book(file_path)
        sht = wb.sheets["Enter Focus List Here"]
        df = CP_df[CP_df['Account Name']==sch_ref_df['School'][sch_ref_df['cyconnect Folder'] == x].values[0]]
        df.drop_duplicates(subset='Student Id', inplace=True)
        sht.range('A3:E300').value = "."
        sht.range('A3').options(index=False, header=False).value = df
        wb.save()
    except:
        print("Failed: " + x)
        continue
    wb.close()

#x = 'Chicago Gale'
#try:
#    file_path = ("Z:\\" + x + "\\" + "SY18 Attendance Tracker - "
#                 + sch_ref_df['Informal School Name'][sch_ref_df['cyconnect Folder'] == x].values[0] + ".xlsx")
#    wb = xw.Book(file_path)
#    sht = wb.sheets["Enter Focus List Here"]
#    df = CP_df[CP_df['Account Name']==sch_ref_df['School'][sch_ref_df['cyconnect Folder'] == x].values[0]]
#    df.drop_duplicates(subset='Student Id', inplace=True)
#    sht.range('A3:E300').value = "."
#    sht.range('A3').options(index=False, header=False).value = df
#    wb.save()
#except:
#    print("Failed: " + x)
#    #continue
#wb.close()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
